In [4]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping


from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle
from keras.layers import Merge

/home/louis_jeay/downloads/3-5-2-test/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/louis_jeay/downloads/3-5-2-test/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys

In [6]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

def load_data(nb_train = 165000,nb_val = 6000,nb_test = 5000):
    # load your data using this function
    verif = []

    verif = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            

    neg1 = verif

    verif=[]

    verif = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]

    verif = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]

    # 
    l = len(bdd)
    matrice = np.zeros((l,36,36,1)) #4
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1

    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])    
    
    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])

    labels = bdd[:,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    np.sum(y[:,1])
    return training, y, validation, val_y, testing, true_y

In [7]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 

In [8]:
def save_model(model):
    # serialize model to JSON
    model_json = model.to_json()
    with open("model_double_final.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model_double_final.h5")
    json_file.close()
    print("Saved model to disk")

In [9]:
def perfs_test(model,test,true):
    print('predicting')
            
    predictions = model.predict(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true)
    perfs = calculate_performance(len(predictions), predictions, true)
    
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

In [10]:
from keras.models import model_from_json
def load_model():
    # load json and create model
    json_file = open('model_double_final.json', 'r')
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    json_file.close()
    # load weights into new model
    loaded_model.load_weights("model_double_final.h5")
    print("Loaded model from disk")
    return loaded_model

In [84]:
Data = load_data(150000,5000,20000)

26017
24222
37991
26023
24229
37993


In [ ]:
# test   
testing = 
true_y = 
print('predicting')
predictions = interaction.predict(testing)[:,1]
print(predictions)
for i,nulll in enumerate(predictions):
    predictions[i] = round(predictions[i])
print(predictions,true_y)
perfs = calculate_performance(len(predictions), predictions, true_y)
print("batch_size : ", batch_size)
print("epochs : ",epochs)
print("acc : ", perfs[0])
print("precision : ", perfs[1])
print("sensitivity : ", perfs[2])
print("specificity : ", perfs[3])
print("MCC : ", perfs[4])

In [42]:
interaction.get_weights()

[array([[[-0.31515267,  0.10921433,  0.30611628, ..., -0.6682983 ,
           0.284731  , -0.3460957 ],
         [ 0.01848096,  0.24445687, -0.24685946, ..., -0.70506537,
          -0.75246817, -0.23083529],
         [-0.3779574 , -0.1279294 ,  0.14173886, ...,  0.30779713,
          -0.2215267 ,  0.42126188],
         [ 0.26697972,  0.3077011 ,  0.27344796, ..., -0.17128827,
           0.29153112,  0.2614919 ]],
 
        [[ 0.3158297 , -0.22502226, -0.12623042, ...,  0.3604244 ,
           0.25494626,  0.15834163],
         [ 0.00260836,  0.14118545, -0.06535099, ...,  0.1476159 ,
          -0.3229163 ,  0.3834469 ],
         [ 0.15092982, -0.49354777, -0.053472  , ..., -0.37413248,
           0.32523882,  0.11519072],
         [-0.48697677,  0.25041163,  0.5165356 , ...,  0.02198852,
          -0.12749082,  0.01619656]],
 
        [[ 0.39626157,  0.37830696, -0.31200373, ..., -0.04240261,
          -0.3479834 ,  0.39330617],
         [-1.4375907 ,  0.06911595,  0.02783217, ...,  0.1

In [71]:
# paramètres 
#true_y = Data[5]
#training = Data[0]
#y = Data[1]
batch_size=48
epochs=25
verbose1 = 1
verbose2 = 1
#validation = Data[2]
#val_y = Data[3]
#testing = Data[4]
matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 64
kernelsize = 7
nbfilters2 = 64
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 0

from keras.layers import merge
from keras import Model
k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.2)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)

c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d2_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.2)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)

c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)
#cnn1d1 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d1_input)
c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)
#cnn1d2 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d2_input)
c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)

c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)

model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.3)(model21)

model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.3)(model22)

model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.3)(model2)

model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.2)(model)

if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
    model = Dropout(0.2)(model)

model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

interaction.load_weights("model_finaldeep.h5")
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#interaction.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose = verbose1)

#print('model training')
#interaction.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])

### Interprétation

In [89]:
interaction.layers[4].name

'conv1d_17'

In [75]:
#interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

In [76]:
#interaction.load_weights("model_finaldeep.h5")

In [74]:
nb_classes = 2
category_index = 1
target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
interaction_output = Lambda(target_layer, output_shape = target_category_loss_output_shape)(interaction_output)
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])
interaction.load_weights("model_finaldeep.h5")

In [93]:
[l.name for l in interaction.layers if l.name == 'conv1d_17']

['conv1d_17']

In [117]:
interaction.input

[<tf.Tensor 'input_25:0' shape=(?, 36, 36, 1) dtype=float32>,
 <tf.Tensor 'input_26:0' shape=(?, 36, 36, 1) dtype=float32>,
 <tf.Tensor 'input_27:0' shape=(?, 36, 4) dtype=float32>,
 <tf.Tensor 'input_28:0' shape=(?, 36, 4) dtype=float32>,
 <tf.Tensor 'input_29:0' shape=(?, 36, 4) dtype=float32>,
 <tf.Tensor 'input_30:0' shape=(?, 36, 4) dtype=float32>]

In [133]:
indice_1 = 1
sequence = Data[0][0][indice_1]
matrice_sequence0 = np.zeros((1,36,36,1))
matrice_sequence0[0,:,:,:] = sequence
sequence = Data[0][1][indice_1]
matrice_sequence1 = np.zeros((1,36,36,1))
matrice_sequence1[0,:,:,:] = sequence
sequence = Data[0][2][indice_1]
matrice_sequence = np.zeros((1,36,4))
matrice_sequence[0,:,:] = sequence
sequence = Data[0][3][indice_1]
matrice_sequence2 = np.zeros((1,36,4))
matrice_sequence2[0,:,:] = sequence
sequence = Data[0][3][indice_1]
matrice_sequence3 = np.zeros((1,36,4))
matrice_sequence3[0,:,:] = sequence
sequence = Data[0][4][indice_1]
matrice_sequence4 = np.zeros((1,36,4))
matrice_sequence4[0,:,:] = sequence

layer_name = 'conv1d_17'
loss = K.sum(interaction.layers[-1].output)
conv_output = [l for l in interaction.layers if l.name == layer_name][0].output
grads = normalize(K.gradients(loss, conv_output)[0])
gradient_function = K.function(interaction.input, [conv_output, grads])

output, grads_val = gradient_function([matrice_sequence0,matrice_sequence1,matrice_sequence,matrice_sequence2,matrice_sequence3,matrice_sequence4])
output, grads_val = output[0, :], grads_val[0, :, :]

weights = np.mean(grads_val, axis = 0)
cam = np.ones(output.shape[0 : 1], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, i]

#cam = cv2.resize(cam, (30, 4))
cam = np.maximum(cam, 0)
heatmap = cam / np.max(cam)

#Return to BGR [0..255] from the preprocessed image
#image = image[0, :]
#image -= np.min(image)
#image = np.minimum(image, 255)

#cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
#cam = np.float32(cam) + np.float32(image)
#cam = 255 * cam / np.max(cam)
cam, heatmap

(array([3.8260002 , 2.9262233 , 3.9561973 , 0.7328272 , 0.        ,
        2.0367832 , 0.5225471 , 0.2763853 , 0.        , 2.0008485 ,
        2.2685115 , 0.71298313, 0.        , 1.8187593 , 2.190754  ,
        2.8255336 , 2.8019283 , 4.211951  , 3.774144  , 0.2751289 ,
        3.1165943 , 3.1165943 , 3.1165943 , 3.1165943 , 3.1165943 ,
        3.1165943 , 3.1165943 , 3.1165943 , 3.1165943 , 3.1165943 ],
       dtype=float32),
 array([0.90836775, 0.694743  , 0.9392791 , 0.1739876 , 0.        ,
        0.48357242, 0.12406297, 0.06561931, 0.        , 0.47504082,
        0.5385893 , 0.16927622, 0.        , 0.43180925, 0.5201281 ,
        0.6708373 , 0.6652329 , 1.        , 0.89605606, 0.06532101,
        0.7399408 , 0.7399408 , 0.7399408 , 0.7399408 , 0.7399408 ,
        0.7399408 , 0.7399408 , 0.7399408 , 0.7399408 , 0.7399408 ],
       dtype=float32))

In [131]:
output.shape

(30, 64)

In [57]:
def target_category_loss_output_shape(input_shape):
    return input_shape

In [58]:
def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_image(path):
    img_path = sys.argv[1]
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)

def compile_saliency_function(model, activation_layer='block5_conv3'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])


### Guided

In [30]:
# modifiy backpropagation
g = tf.get_default_graph()
name = 'GuidedBackProp'
with g.gradient_override_map({'Relu': name}):

    # get layers that have an activation
    layer_dict = [layer for layer in interaction.layers[1:] if hasattr(layer, 'activation')]

    # replace relu activation
    for layer in layer_dict:
        if layer.activation == keras.activations.relu:
            layer.activation = tf.nn.relu